<a href="https://colab.research.google.com/github/MxFxM/ColabCodes/blob/master/GPS_Logging_Auswertung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount drive and test for success

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

with open(f"/content/gdrive/My Drive/GPSLogger for Android/gpslogger_test.xml", "r") as json_file:
    for line in json_file:
        print(line)
        break

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
<x>This is a test file</x>


Populate databank with ?new? data.
To confirm that it is indeed new data, please check the date before running. The code should however, not create double entries.

In [0]:
FILE_DATES = [n for n in range(20190706, 20190732)]
print(FILE_DATES)

[20190706, 20190707, 20190708, 20190709, 20190710, 20190711, 20190712, 20190713, 20190714, 20190715, 20190716, 20190717, 20190718, 20190719, 20190720, 20190721, 20190722, 20190723, 20190724, 20190725, 20190726, 20190727, 20190728, 20190729, 20190730, 20190731]


In [0]:
import json
from datetime import datetime
import time
import sqlite3

GEOJSON_DATE_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"
FILE_DATES = [n for n in range(20190706, 20190732)]
# last date was 2019 07 31

sql_transaction = []
DATABANK_PATH = "/content/gdrive/My Drive/GPSLogger for Android/gpslogs.db"
connection = sqlite3.connect(DATABANK_PATH)
c = connection.cursor()

def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS gpslogs(time_long INT PRIMARY KEY UNIQUE, year INT, month INT, day INT, hour INT, minute INT, second REAL, accuracy REAL, altitude REAL, bearing REAL, speed REAL, latitude REAL, longitude REAL)")

def transaction_bldr(sql, write=False):
    global sql_transaction
    if not write:
        sql_transaction.append(sql)
    if write:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except Exception as e:
                print(f"Error during writing: {e}")
        connection.commit()
        sql_transaction = []

def sql_insert(tl, ye, mo, da, ho, mi, se, ac, al, be, sp, la, lo):
    try:
        sql=f"INSERT INTO gpslogs (time_long, year, month, day, hour, minute, second, accuracy, altitude, bearing, speed, latitude, longitude) VALUES ({int(tl)}, {int(ye)}, {int(mo)}, {int(da)}, {int(ho)}, {int(mi)}, {float(se)}, {float(ac)}, {float(al)}, {float(be)}, {float(sp)}, {float(la)}, {float(lo)});"
        transaction_bldr(sql)
    except Exception as e:
        print(f"Error on inserting: {e}")

# DEBUG variables, not strictly necessary
printfirst = True
lensum = 0
starttime = time.time()

create_table()

for FILE_DATE in FILE_DATES:
    with open(f"/content/gdrive/My Drive/GPSLogger for Android/{FILE_DATE}.geojson", "r") as json_file:
        data = json.load(json_file)
        for feature in data['features']:
            # one feature corresponds to one measurement
            if 'properties' in feature:
                properties = feature['properties']
                if 'provider' in properties:
                    pr = properties['provider']
                else:
                    pr = "None"
                if 'time_long' in properties:
                    tl = properties['time_long']
                else:
                    break # time is crucial
                if 'accuracy' in properties:
                    ac = properties['accuracy']
                else:
                    ac = 100.0
                if 'altitude' in properties:
                    al = properties['altitude']
                else:
                    break # position is crucial
                if 'bearing' in properties:
                    be = properties['bearing']
                else:
                    be = 0.0
                if 'speed' in properties:
                    sp = properties['speed']
                else:
                    sp = 0.0

                if 'time' in properties:
                    datetime = datetime.strptime(properties['time'], GEOJSON_DATE_FORMAT)
                    ye = datetime.year
                    mo = datetime.month
                    da = datetime.day
                    ho = datetime.hour
                    mi = datetime.minute
                    se = datetime.second
                else:
                    break # time is crucial
                
            if 'geometry' in feature:
                geometry = feature['geometry']
                if 'coordinates' in geometry:
                    la = geometry['coordinates'][1]
                    lo = geometry['coordinates'][0]
                else:
                    break # position is crucial

            """
            if printfirst:
                printfirst = False
                print(feature)
                print(f"Time: {datetime}")
                print(f"Year: {datetime.year}")
                print(f"Month: {datetime.month}")
                print(f"Day: {datetime.day}")
                print(f"Hour: {datetime.hour}")
                print(f"Minute: {datetime.minute}")
                print(f"Second: {datetime.second}")
                print(f"Provider: {properties['provider']}")
                print(f"Time (long): {properties['time_long']}")
                print(f"Accuracy: {properties['accuracy']}")
                print(f"Altitude: {properties['altitude']}")
                print(f"Bearing: {properties['bearing']}")
                print(f"Speed: {properties['speed']}")
                print(f"Latitude: {geometry['coordinates'][1]}")
                print(f"Longitude: {geometry['coordinates'][0]}")
            """
            
            sql_insert(tl, ye, mo, da, ho, mi, se, ac, al, be, sp, la, lo)
            
            """
            sql_insert(properties['time_long'],
                       datetime.year,
                       datetime.month,
                       datetime.day,
                       datetime.hour,
                       datetime.minute,
                       datetime.second,
                       properties['accuracy'],
                       properties['altitude'],
                       properties['bearing'],
                       properties['speed'],
                       geometry['coordinates'][1],
                       geometry['coordinates'][0])
            """

        transaction_bldr("", write=True)
        lensum = lensum + len(data['features'])

connection.close()
        
print()
print(f"Processed {lensum} data structures from {len(FILE_DATES)} file(s)")
print(f"Processing took {time.time() - starttime} seconds")


Processed 2851 data structures from 26 file(s)
Processing took 9.637007236480713 seconds
